In [1]:
import Pkg; Pkg.instantiate()
Pkg.activate(".")

  Activating project at `~/DrugResponseModel.jl`


In [2]:
using DelimitedFiles
using Plots
using Measures
using LinearAlgebra
using Statistics
using DSP: conv
using BlackBoxOptim
using JLD
using Distributions
using DataFrames
using XLSX
using StatsPlots
using NumericalIntegration
using CSV
using MultivariateStats
using Impute

In [3]:
using DrugResponseModel

In [4]:
using Plots.PlotMeasures 
gr()

Plots.GRBackend()

In [12]:
concs, popul1, g1s1, g2s1 = DrugResponseModel.load(189, 1)
_, popul2, g1s2, g2s2 = DrugResponseModel.load(189, 2)
_, popul3, g1s3, g2s3 = DrugResponseModel.load(189, 3)

concs

8×5 Matrix{Float64}:
   0.0    0.0    0.0    0.0    0.0
   5.0    1.0    0.25   0.1    5.0
  10.0   10.0    1.0    1.0   10.0
  25.0   20.0    2.5    2.0   25.0
  50.0   50.0    5.0    3.0   50.0
 100.0  125.0   10.0    5.0  100.0
 250.0  250.0   30.0    7.5  250.0
 500.0  500.0  100.0   15.0  500.0

In [13]:
# find the average of the three replicates
g1S = cat(g1s1, g1s2, g1s3, dims = 4)
g2S = cat(g2s1, g2s2, g2s3, dims = 4)
g1m = mean(g1S, dims = 4) # mean G1
g2m = mean(g2S, dims = 4) # mean G2

time = LinRange(0.0, 95.0, 189)

# This is a 98-long vector containing parameters of the first 5 drugs AU565 treatment.
ps = DrugResponseModel.parameters()
efcs = getODEparams(ps, concs)

16×8×5 Array{Float64, 3}:
[:, :, 1] =
 1.99713   1.88725     1.74199      …  0.673886    0.465172    0.403194
 0.10733   0.101515    0.093827        0.0372982   0.0262522   0.022972
 1.99997   1.86566     1.68811         0.38254     0.127425    0.0516668
 0.988148  0.922374    0.835426        0.196071    0.0711379   0.0340383
 1.74038   1.6431      1.5145          0.568895    0.384118    0.329247
 1.99998   1.86567     1.68812      …  0.382546    0.127429    0.051671
 0.20115   0.20531     0.210809        0.251245    0.259146    0.261493
 1.84866   1.84974     1.85116         1.86162     1.86367     1.86428
 0.0       0.00428769  0.00995564      0.0516338   0.0597779   0.0621964
 0.0       3.53381e-8  8.2052e-8       4.25553e-7  4.92676e-7  5.12608e-7
 0.0       0.00239883  0.00556988   …  0.0288875   0.0334439   0.034797
 0.0       2.27417e-8  5.28042e-8      2.73863e-7  3.17059e-7  3.29886e-7
 0.0       1.15495e-6  2.68169e-6      1.39083e-5  1.6102e-5   1.67534e-5
 0.0       2.82061

In [14]:
#lapatinib
#lapatinib_cost, lapatinib_hill = optimize_hill(concs[:, 1], g1m[:, :, 1], g2m[:, :, 1])

lapatinib_efcs = efcs[:, :, 1]

16×8 Matrix{Float64}:
 1.99713   1.88725     1.74199      …  0.673886    0.465172    0.403194
 0.10733   0.101515    0.093827        0.0372982   0.0262522   0.022972
 1.99997   1.86566     1.68811         0.38254     0.127425    0.0516668
 0.988148  0.922374    0.835426        0.196071    0.0711379   0.0340383
 1.74038   1.6431      1.5145          0.568895    0.384118    0.329247
 1.99998   1.86567     1.68812      …  0.382546    0.127429    0.051671
 0.20115   0.20531     0.210809        0.251245    0.259146    0.261493
 1.84866   1.84974     1.85116         1.86162     1.86367     1.86428
 0.0       0.00428769  0.00995564      0.0516338   0.0597779   0.0621964
 0.0       3.53381e-8  8.2052e-8       4.25553e-7  4.92676e-7  5.12608e-7
 0.0       0.00239883  0.00556988   …  0.0288875   0.0334439   0.034797
 0.0       2.27417e-8  5.28042e-8      2.73863e-7  3.17059e-7  3.29886e-7
 0.0       1.15495e-6  2.68169e-6      1.39083e-5  1.6102e-5   1.67534e-5
 0.0       2.82061e-8  6.54921e-8 

In [8]:
# converting the Hill parameters to ODE parameters
#lapatinib_hill_ode = getODEparams(lapatinib_hill, concs[:, 1])

In [33]:
lapatinib_G1_control = zeros(189, 8) # model prediction of cell numbers for all 8 concentrations of lapatinib
lapatinib_G2_control = zeros(189, 8)

#for i = 1:8
#    lapatinib_G1[:, i], lapatinib_G2[:, i], _ = predict(lapatinib_hill_ode[:, i, 1], lapatinib_hill_ode[:, 1, 1], time)
#end


#default: nG1 = 8, nG2 = 20  
for i = 1:8
    lapatinib_G1_control[:, i], lapatinib_G2_control[:, i], _ = 
    DrugResponseModel.predict(lapatinib_efcs[:, i, 1], lapatinib_efcs[:, 1, 1], time)
end

lapatinib_G1_control

189×8 Matrix{Float64}:
 0.0128326  0.0128326  0.0128326  …  0.0128326  0.0128326   0.0128326
 0.0129165  0.0133641  0.0139874     0.0199223  0.0214257   0.0218984
 0.013001   0.0136271  0.0145288     0.0249578  0.0281519   0.0292038
 0.013086   0.0137977  0.014843      0.0287346  0.0336462   0.0353287
 0.0131715  0.013945   0.015092      0.0318284  0.0384262   0.0407641
 0.0132576  0.0140894  0.0153283  …  0.0345422  0.0427728   0.0457755
 0.0133443  0.0142317  0.0155565     0.0369745  0.046765    0.0504287
 0.0134316  0.0143683  0.015769      0.0391272  0.0503825   0.0546903
 0.0135194  0.0144965  0.0159598     0.0409838  0.053588    0.0585103
 0.0136078  0.0146162  0.0161285     0.042544   0.0563659   0.0618635
 0.0136968  0.0147287  0.0162778  …  0.0438284  0.0587298   0.0647567
 0.0137863  0.0148356  0.0164122     0.0448715  0.0607145   0.0672207
 0.0138765  0.0149388  0.016536      0.0457125  0.0623646   0.0692992
 ⋮                                ⋱  ⋮                      
 0.040

In [32]:
lapatinib_G1_reduced = zeros(189, 8) 
lapatinib_G2_reduced = zeros(189, 8)

#both nG1 and nG2 reduced: nG1 = 4, nG2 = 12  
for i = 1:8
    lapatinib_G1_reduced[:, i], lapatinib_G2_reduced[:, i], _ = 
    DrugResponseModel.predict(lapatinib_efcs[:, i, 1], lapatinib_efcs[:, 1, 1], time, 4, 12)
end

lapatinib_G1_reduced

189×8 Matrix{Float64}:
 0.0233876  0.0233876  0.0233876  …  0.0233876  0.0233876  0.0233876
 0.0236685  0.0244949  0.0256452     0.0365836  0.0393513  0.0402213
 0.0239528  0.0251598  0.0268949     0.0467744  0.0528187  0.0548061
 0.0242406  0.0256835  0.0277954     0.0553337  0.0649256  0.0682002
 0.0245318  0.0261468  0.0285343     0.0626391  0.0758398  0.0804958
 0.0248264  0.026568   0.0291581  …  0.0687304  0.0854154  0.0914743
 0.0251247  0.0269552  0.0296877     0.0736361  0.0935392  0.100961
 0.0254265  0.0273158  0.0301429     0.0774425  0.100204   0.1089
 0.0257319  0.0276573  0.0305419     0.0802736  0.105481   0.115332
 0.026041   0.0279853  0.0308991     0.082261   0.109483   0.120344
 0.0263538  0.0283038  0.0312251  …  0.0835259  0.11233    0.124049
 0.0266704  0.0286154  0.0315264     0.0841727  0.114143   0.12656
 0.0269908  0.0289218  0.0318076     0.0842894  0.115033   0.127989
 ⋮                                ⋱  ⋮                     
 0.193589   0.190257   0.18411

In [34]:
lapatinib_G1_doubled = zeros(189, 8) 
lapatinib_G2_doubled = zeros(189, 8)


#both nG1 and nG2 doubled: nG1 = 16, nG2 = 40  
for i = 1:8
    lapatinib_G1_doubled[:, i], lapatinib_G2_doubled[:, i], _ = 
    DrugResponseModel.predict(lapatinib_efcs[:, i, 1], lapatinib_efcs[:, 1, 1], time, 16, 40)
end

lapatinib_G1_doubled

189×8 Matrix{Float64}:
 0.00632985  0.00632985  0.00632985  …  0.00632985  0.00632985  0.00632985
 0.00635016  0.00657048  0.00687724     0.00979879  0.010539    0.0107717
 0.00637054  0.00667627  0.00711679     0.0122205   0.0137857   0.0143014
 0.00639099  0.0067303   0.00722942     0.0139167   0.0162948   0.0171104
 0.0064115   0.00676464  0.00729018     0.015113    0.0182414   0.0193534
 0.00643208  0.00679216  0.00733123  …  0.0159736   0.0197698   0.0211629
 0.00645272  0.0068186   0.00736782     0.0166252   0.0210062   0.0226601
 0.00647343  0.00684693  0.007408       0.0171677   0.0220619   0.0239562
 0.0064942   0.00687851  0.00745566     0.0176738   0.023027    0.0251438
 0.00651504  0.00691343  0.00751134     0.0181871   0.0239617   0.0262872
 0.00653595  0.0069508   0.00757306  …  0.0187229   0.0248946   0.0274183
 0.00655693  0.00698922  0.00763749     0.0192753   0.0258273   0.0285407
 0.00657797  0.00702729  0.00770114     0.0198261   0.0267436   0.0296393
 ⋮            